In [4]:
!pip install catboost
!pip install mljar-supervised
!pip install transformers
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!python -V

Python 3.10.11


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import AutoModelForAudioClassification, Wav2Vec2FeatureExtractor
import torch
import torch.nn as nn
from torchaudio import transforms
import torchaudio
from torch.utils.data import Dataset, DataLoader
from scipy import stats
from autogluon.tabular import TabularDataset, TabularPredictor

import torchaudio.transforms as transforms
from sklearn.metrics import accuracy_score
import librosa
import librosa.display as dsp
from IPython.display import Audio
from tqdm import tqdm
import random
import os
import numpy as np
import pandas as pd
from supervised.automl import AutoML
from torch.nn.utils.rnn import pad_sequence

import glob
import re
import seaborn as sns
import unicodedata
import matplotlib
import matplotlib.pyplot as plt
#import koreanize_matplotlib
import matplotlib.font_manager as fm

In [7]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # Melspectrogram 벡터를 추출할 개수
    'SEED':42,
    'BATCH_SIZE':16,
    'LEARNING_RATE':1e-04,
    'NUM_CLASSES':6,
    'INPUT_SIZE':1,
    'HIDDEN_SIZE':32,
    'NUM_LAYERS':3,
    'EPOCHS':10
}

In [10]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED'])

In [11]:
path = '/content/gdrive/MyDrive/sound/open'
train = pd.read_csv(f'{path}/train.csv')
test = pd.read_csv(f'{path}/test.csv')
ans = f'{path}/sample_submission.csv'
submit = pd.read_csv(ans)

In [12]:
train.head()

,id,path,label
0,TRAIN_0000,./train/TRAIN_0000.wav,1
1,TRAIN_0001,./train/TRAIN_0001.wav,2
2,TRAIN_0002,./train/TRAIN_0002.wav,4
3,TRAIN_0003,./train/TRAIN_0003.wav,5
4,TRAIN_0004,./train/TRAIN_0004.wav,4


In [13]:
train['label'].value_counts()

0    867
2    859
5    853
3    852
1    848
4    722
Name: label, dtype: int64

In [14]:
test.head()

,id,path
0,TEST_0000,./test/TEST_0000.wav
1,TEST_0001,./test/TEST_0001.wav
2,TEST_0002,./test/TEST_0002.wav
3,TEST_0003,./test/TEST_0003.wav
4,TEST_0004,./test/TEST_0004.wav


In [15]:
test

,id,path
0,TEST_0000,./test/TEST_0000.wav
1,TEST_0001,./test/TEST_0001.wav
2,TEST_0002,./test/TEST_0002.wav
3,TEST_0003,./test/TEST_0003.wav
4,TEST_0004,./test/TEST_0004.wav
...,...,...
1876,TEST_1876,./test/TEST_1876.wav
1877,TEST_1877,./test/TEST_1877.wav
1878,TEST_1878,./test/TEST_1878.wav
1879,TEST_1879,./test/TEST_1879.wav


In [16]:
test['path'][0][1:]

'/test/TEST_0000.wav'

In [17]:
real_path = path
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['path']):
        path = real_path+path[1:]
        y, sr = librosa.load(path, sr=CFG['SR'])
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
        y_feature = []
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)

    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    return mfcc_df

In [20]:
def get_feature_mel(df):
    features = []
    for path in tqdm(df['path']):
        path = real_path+path[1:]
        data, sr = librosa.load(path, sr=CFG['SR'])
        n_fft = 2048
        win_length = 2048
        hop_length = 1024
        n_mels = 128

        D = np.abs(librosa.stft(data, n_fft=n_fft, win_length = win_length, hop_length=hop_length))
        mel = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=n_mels, hop_length=hop_length, win_length=win_length)

        features.append({
            'mel_mean': mel.mean(axis=1),
            'mel_max': mel.min(axis=1),
            'mel_min': mel.max(axis=1),
        })
    mel_df = pd.DataFrame(features)
    mel_mean_df = pd.DataFrame(mel_df['mel_mean'].tolist(), columns=[f'mel_mean_{i}' for i in range(n_mels)])
    mel_max_df = pd.DataFrame(mel_df['mel_max'].tolist(), columns=[f'mel_max_{i}' for i in range(n_mels)])
    mel_min_df = pd.DataFrame(mel_df['mel_min'].tolist(), columns=[f'mel_min_{i}' for i in range(n_mels)])

    return pd.concat([mel_mean_df, mel_max_df, mel_min_df], axis=1)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(train, train['label'], test_size=0.2, random_state=CFG['SEED'])

In [21]:
mfcc_train_x = get_mfcc_feature(train)
mfcc_test_x = get_mfcc_feature(test)
mel_train_x = get_feature_mel(train)
mel_test_x = get_feature_mel(test)

100%|██████████| 1881/1881 [00:29<00:00, 63.05it/s]


In [79]:
train_x = pd.concat([mel_train_x, mfcc_train_x], axis=1)
test_x = pd.concat([mel_test_x, mfcc_test_x], axis=1)

In [85]:
scaler = StandardScaler()
Z_data = scaler.fit_transform(train_x)
Z_datat = scaler.fit_transform(test_x)

In [87]:
test_x

,mel_mean_0,mel_mean_1,mel_mean_2,mel_mean_3,mel_mean_4,mel_mean_5,mel_mean_6,mel_mean_7,mel_mean_8,mel_mean_9,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,0.077431,0.045913,0.067294,0.119950,0.152970,0.122273,0.131134,0.171050,0.229612,0.268162,...,-0.452362,-1.410518,-2.177119,0.198908,1.068798,0.964545,-2.613074,2.812386,0.901755,1.809820
1,0.080825,0.057177,0.069734,0.089675,0.070371,0.062837,0.077833,0.103250,0.140345,0.254296,...,1.778805,2.596544,-1.626949,0.363649,-1.242502,-1.523350,0.161729,2.599890,0.549056,-0.346372
2,0.014955,0.045311,0.097314,0.119791,0.157356,0.071188,0.064243,0.077637,0.111111,0.149076,...,-0.988734,-3.927216,-4.373907,-1.453163,-3.116022,0.184532,-3.222616,-0.724787,-1.841568,-4.278657
3,0.088075,0.053243,0.065644,0.096410,0.105379,0.143737,0.153011,0.226024,0.236106,0.156049,...,-1.371162,-1.021333,-3.315774,-2.259278,-1.717189,-0.019883,-0.051858,0.199551,2.846354,5.373621
4,0.089540,0.058359,0.060024,0.111930,0.130118,0.261876,0.231911,0.146480,0.155684,0.258896,...,2.007670,-1.133699,1.177092,-0.153630,-1.755448,1.917315,-2.084665,-1.174302,-0.747732,-1.348814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,0.094810,0.069532,0.075272,0.100861,0.089328,0.065800,0.084794,0.091190,0.121290,0.173096,...,1.999792,-3.934587,-0.296250,-0.949789,-0.430385,-2.024839,-3.166185,3.210046,2.162896,1.188407
1877,0.076132,0.062607,0.071826,0.104158,0.105444,0.074122,0.086256,0.124237,0.180672,0.199906,...,0.566558,5.903645,3.492442,1.928877,0.612623,-0.503881,0.614712,3.338864,1.078280,1.294159
1878,0.010398,0.017304,0.035367,0.051940,0.087565,0.192930,0.205463,0.137716,0.147201,0.117158,...,-4.568923,-1.418632,-6.151373,-2.744539,-3.983426,-5.843061,-1.557026,1.383257,0.714762,-2.767536
1879,0.014691,0.042166,0.072805,0.086481,0.110771,0.058583,0.077228,0.088876,0.115158,0.168497,...,2.544887,-1.345986,-1.827731,-2.109568,-2.891415,-0.612320,-2.212980,-1.055952,-1.289654,-1.245360


### Testing

In [24]:
X_train, X_test, y_train, y_test = train_test_split(train, train['label'], test_size=0.2, random_state=CFG['SEED'])

In [28]:
mfcc_X_train = get_mfcc_feature(X_train)
mfcc_X_test = get_mfcc_feature(X_test)
# mfcc_y_train = get_mfcc_feature(y_train)
# mfcc_y_test = get_mfcc_feature(y_test)
mel_X_train = get_feature_mel(X_train)
mel_X_test = get_feature_mel(X_test)
# mel_y_train = get_feature_mel(y_train)
# mel_y_test = get_feature_mel(y_test)

100%|██████████| 1001/1001 [00:19<00:00, 50.12it/s]


In [29]:
X_train = pd.concat([mel_X_train, mfcc_X_train], axis=1)
X_test = pd.concat([mel_X_test, mfcc_X_test], axis=1)

### RandomFC

In [36]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer


In [30]:
model = RandomForestClassifier(random_state=CFG['SEED'], verbose=1)

In [37]:
#Z-Score
scaler = StandardScaler()
Z_data = scaler.fit_transform(X_train)
Z_datat = scaler.fit_transform(X_test)

#Min-Max
scaler = MinMaxScaler()
MM_data = scaler.fit_transform(X_train)
MM_datat = scaler.fit_transform(X_test)

ZMM_data = scaler.fit_transform(Z_data)
ZMM_datat = scaler.fit_transform(Z_datat)

#Unit-Length
scaler = Normalizer(norm='max')
UL_data = scaler.fit_transform(X_train)
UL_datat = scaler.fit_transform(X_test)

#로그변환 Log Transformation
log_data = np.log(X_train)
log_datat = np.log(X_test)


/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log


In [ ]:
scaler = StandardScaler()
Z_data = scaler.fit_transform(train_x)
Z_datat = scaler.fit_transform(test_x)

In [44]:
model.fit(ZMM_data, y_train)
#model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   11.1s finished


RandomForestClassifier(random_state=42, verbose=1)

In [45]:
#y_pred = model.predict(X_test)
y_pred = model.predict(ZMM_datat)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [46]:
# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)#0.43794
#Accuracy: 0.45054945054945056
#Accuracy: 0.3246753246753247
#Accuracy: 0.4115884115884116

#Accuracy: 0.46053946053946054


Accuracy: 0.42357642357642356


In [ ]:
submission = pd.read_csv(ans)
submission['label'] = y_pred
submission.to_csv('./baseline_submission.csv', index=False)

### SVM

In [ ]:
from sklearn import svm

In [ ]:
model = svm.SVC(random_state=CFG['SEED'], verbose=True)

In [ ]:
model.fit(train_x, y_train)

In [ ]:
y_pred = model.predict(test_x)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)#0.43794

### Dicision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(random_state=CFG['SEED'])
model.fit(train_x, y_train)

In [ ]:
y_pred = model.predict(test_x)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)#0.43794

### CatBoost

In [47]:
import catboost
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer


In [ ]:
#train_x, y_train

In [ ]:
train_y

0       1
1       2
2       4
3       5
4       4
       ..
4996    5
4997    0
4998    1
4999    1
5000    4
Name: label, Length: 5001, dtype: int64

In [ ]:
#정규화

In [ ]:
#Z-Score
scaler = StandardScaler()
Z_data = scaler.fit_transform(tx)
Z_datat = scaler.fit_transform(xt)

#Min-Max
scaler = MinMaxScaler()
MM_data = scaler.fit_transform(tx)
MM_datat = scaler.fit_transform(xt)

ZMM_data = scaler.fit_transform(Z_data)
ZMM_datat = scaler.fit_transform(Z_datat)

#Unit-Length
scaler = Normalizer(norm='max')
UL_data = scaler.fit_transform(tx)
UL_datat = scaler.fit_transform(xt)

#로그변환 Log Transformation
log_data = np.log(tx)
log_datat = np.log(xt)

/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log


In [58]:
scaler = StandardScaler()
Z_data = scaler.fit_transform(train_x)
Z_datat = scaler.fit_transform(test_x)

In [61]:
catboost_model = CatBoostClassifier()
#catboost_model.fit(UL_data, y_train, eval_set=(UL_datat, y_test), verbose=True)
catboost_model.fit(Z_data, train_y, verbose=True)

#bagging_model = BaggingClassifier(base_estimator=catboost_model, n_estimators=10)

Learning rate set to 0.085896
0:	learn: 1.7393399	total: 1.23s	remaining: 20m 24s
1:	learn: 1.6965952	total: 2.26s	remaining: 18m 49s
2:	learn: 1.6625457	total: 3.27s	remaining: 18m 6s
3:	learn: 1.6316659	total: 4.14s	remaining: 17m 10s
4:	learn: 1.6045397	total: 4.98s	remaining: 16m 31s
5:	learn: 1.5813431	total: 5.84s	remaining: 16m 8s
6:	learn: 1.5594720	total: 6.39s	remaining: 15m 6s
7:	learn: 1.5386090	total: 6.91s	remaining: 14m 17s
8:	learn: 1.5213301	total: 7.46s	remaining: 13m 41s
9:	learn: 1.5059435	total: 7.97s	remaining: 13m 9s
10:	learn: 1.4927061	total: 8.52s	remaining: 12m 45s
11:	learn: 1.4789540	total: 9.19s	remaining: 12m 36s
12:	learn: 1.4677148	total: 10s	remaining: 12m 40s
13:	learn: 1.4562920	total: 10.9s	remaining: 12m 46s
14:	learn: 1.4464400	total: 11.7s	remaining: 12m 48s
15:	learn: 1.4373538	total: 12.3s	remaining: 12m 35s
16:	learn: 1.4285690	total: 12.8s	remaining: 12m 20s
17:	learn: 1.4202013	total: 13.3s	remaining: 12m 6s
18:	learn: 1.4129248	total: 13.8s

In [62]:
y_pred = catboost_model.predict(Z_datat)

In [ ]:
# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print("Validation Accuracy:", accuracy)
#Validation Accuracy: 0.47952047952047955
#Validation Accuracy: 0.2897102897102897
#Validation Accuracy: 0.28471528471528473
#Validation Accuracy: 0.44555444555444557

#Validation Accuracy: 0.4965034965034965

In [64]:
submission = pd.read_csv(ans)
submission['label'] = y_pred
submission.to_csv('./baseline_submission.csv', index=False)

In [ ]:
#audio normalization
def level_normalization(audio, target_dBFS):
    rms = np.sqrt(np.mean(np.square(audio)))
    current_dBFS = 20 * np.log10(rms)

    dB_difference = target_dBFS - current_dBFS

    normalized_audio = audio * (10 ** (dB_difference / 20))

    return normalized_audio

def clipping_prevention(audio, threshold):
    max_amp = np.max(np.abs(audio))
    scale_factor = 1.0
    if max_amp > threshold:
        scale_factor = threshold / max_amp

    clipped_audio = audio * scale_factor

    return clipped_audio

def dynamic_range_compression(audio, threshold):
    compressed_audio = np.sign(audio) * np.log1p(threshold * np.abs(audio)) / np.log1p(threshold)

    return compressed_audio

### KFold

In [ ]:
train_x

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,-414.755737,110.100639,46.699074,23.939814,14.766221,4.820827,1.445079,-0.926153,2.892509,4.465990,...,-2.615444,-1.620636,-1.285820,-1.538143,-1.629623,-0.110236,0.022208,3.912367,4.398968,2.815795
1,-399.769531,83.051300,55.473316,31.782587,22.040751,0.985082,3.712759,-2.359319,-0.026561,0.393656,...,1.074466,1.944394,0.017539,-0.097045,0.061731,1.764198,0.311808,2.248236,0.097092,-0.008172
2,-341.145081,97.399071,38.274349,19.811539,0.731027,0.838704,-1.911201,-10.645283,-1.595292,-2.305276,...,-1.521030,1.525717,0.446549,-0.879012,-0.619363,3.587360,4.218765,3.195291,-0.777671,-0.989654
3,-376.963715,118.961670,34.490349,24.178417,-1.065604,-1.613391,2.888372,-7.865876,-2.260846,-5.609926,...,-1.049496,-3.562966,-3.667920,-2.209875,-0.537536,-1.000548,-3.482561,-2.733938,-0.503050,-0.844000
4,-352.863220,117.553337,29.948687,31.094315,5.406391,-5.591997,-4.809399,-11.500416,-6.894948,2.946273,...,-0.323433,0.386383,-3.801969,-2.157103,-0.022309,1.282429,1.643818,2.032168,-0.533625,-1.729753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,-416.181305,112.938484,47.294235,23.111433,16.048231,6.795750,0.194628,-3.153772,-5.016596,-7.414401,...,-1.217834,-2.810278,-3.939800,-4.212123,-1.823620,0.406775,0.249249,0.091639,-1.120364,-1.970424
4997,-237.811432,72.207787,3.371584,14.646128,-10.546066,7.671333,-1.676355,-8.311659,5.224692,-3.619729,...,0.276755,1.673036,-2.383265,-1.866505,-4.395771,-1.290971,-0.243259,-0.217550,0.025550,0.289346
4998,-368.228119,103.887871,40.081413,21.763754,14.120457,5.778781,-3.776188,-4.852784,-3.689435,-0.837068,...,0.427912,1.294595,1.419106,2.330357,1.584116,2.376556,0.746695,3.503191,1.551537,-0.069177
4999,-407.114288,103.880676,44.980690,23.265039,13.279220,6.904114,-1.792800,3.526098,2.991546,-1.412130,...,0.797132,1.962759,0.478440,-1.340471,0.208459,3.158219,2.288530,1.665004,-0.770614,0.064017


In [84]:
test

,id,path
0,TEST_0000,./test/TEST_0000.wav
1,TEST_0001,./test/TEST_0001.wav
2,TEST_0002,./test/TEST_0002.wav
3,TEST_0003,./test/TEST_0003.wav
4,TEST_0004,./test/TEST_0004.wav
...,...,...
1876,TEST_1876,./test/TEST_1876.wav
1877,TEST_1877,./test/TEST_1877.wav
1878,TEST_1878,./test/TEST_1878.wav
1879,TEST_1879,./test/TEST_1879.wav


In [ ]:
t = pd.concat([train_x, train_y], axis=1)
t

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32,label
0,-414.755737,110.100639,46.699074,23.939814,14.766221,4.820827,1.445079,-0.926153,2.892509,4.465990,...,-1.620636,-1.285820,-1.538143,-1.629623,-0.110236,0.022208,3.912367,4.398968,2.815795,1
1,-399.769531,83.051300,55.473316,31.782587,22.040751,0.985082,3.712759,-2.359319,-0.026561,0.393656,...,1.944394,0.017539,-0.097045,0.061731,1.764198,0.311808,2.248236,0.097092,-0.008172,2
2,-341.145081,97.399071,38.274349,19.811539,0.731027,0.838704,-1.911201,-10.645283,-1.595292,-2.305276,...,1.525717,0.446549,-0.879012,-0.619363,3.587360,4.218765,3.195291,-0.777671,-0.989654,4
3,-376.963715,118.961670,34.490349,24.178417,-1.065604,-1.613391,2.888372,-7.865876,-2.260846,-5.609926,...,-3.562966,-3.667920,-2.209875,-0.537536,-1.000548,-3.482561,-2.733938,-0.503050,-0.844000,5
4,-352.863220,117.553337,29.948687,31.094315,5.406391,-5.591997,-4.809399,-11.500416,-6.894948,2.946273,...,0.386383,-3.801969,-2.157103,-0.022309,1.282429,1.643818,2.032168,-0.533625,-1.729753,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,-416.181305,112.938484,47.294235,23.111433,16.048231,6.795750,0.194628,-3.153772,-5.016596,-7.414401,...,-2.810278,-3.939800,-4.212123,-1.823620,0.406775,0.249249,0.091639,-1.120364,-1.970424,5
4997,-237.811432,72.207787,3.371584,14.646128,-10.546066,7.671333,-1.676355,-8.311659,5.224692,-3.619729,...,1.673036,-2.383265,-1.866505,-4.395771,-1.290971,-0.243259,-0.217550,0.025550,0.289346,0
4998,-368.228119,103.887871,40.081413,21.763754,14.120457,5.778781,-3.776188,-4.852784,-3.689435,-0.837068,...,1.294595,1.419106,2.330357,1.584116,2.376556,0.746695,3.503191,1.551537,-0.069177,1
4999,-407.114288,103.880676,44.980690,23.265039,13.279220,6.904114,-1.792800,3.526098,2.991546,-1.412130,...,1.962759,0.478440,-1.340471,0.208459,3.158219,2.288530,1.665004,-0.770614,0.064017,1


In [ ]:
#using KFOLD
from sklearn.model_selection import KFold
#data = train_x, test_x, train_y
kf = KFold(n_splits=10, random_state=69, shuffle=True)
scores = []

for train_index, test_index in kf.split(t):
    train_data = t.iloc[train_index]
    test_data = t.iloc[test_index]

    x_train = train_data.drop('label', axis=1).values
    y_train = train_data['label'].values

    x_val = test_data.drop('label', axis=1).values
    y_val = test_data['label'].values

    model = catboost.CatBoostClassifier()

    model.fit(x_train, y_train, verbose=True)

    y_pred = model.predict(x_val)

    accuracy = accuracy_score(y_val, y_pred)
    scores.append(accuracy)

print("Cross Validation Scores:", scores)
print("Average Accuracy:", np.mean(scores))  

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
6:	learn: 1.5920285	total: 385ms	remaining: 54.6s
7:	learn: 1.5748735	total: 431ms	remaining: 53.5s
8:	learn: 1.5577241	total: 479ms	remaining: 52.7s
9:	learn: 1.5453708	total: 528ms	remaining: 52.2s
10:	learn: 1.5320181	total: 576ms	remaining: 51.8s
11:	learn: 1.5170427	total: 633ms	remaining: 52.1s
12:	learn: 1.5049296	total: 685ms	remaining: 52s
13:	learn: 1.4952774	total: 732ms	remaining: 51.6s
14:	learn: 1.4840529	total: 781ms	remaining: 51.3s
15:	learn: 1.4747478	total: 828ms	remaining: 50.9s
16:	learn: 1.4668803	total: 884ms	remaining: 51.1s
17:	learn: 1.4571551	total: 934ms	remaining: 50.9s
18:	learn: 1.4506963	total: 982ms	remaining: 50.7s
19:	learn: 1.4435009	total: 1.03s	remaining: 50.5s
20:	learn: 1.4359159	total: 1.09s	remaining: 50.8s
21:	learn: 1.4322636	total: 1.15s	remaining: 51s
22:	learn: 1.4250156	total: 1.2s	remaining: 50.8s
23:	learn: 1.4193257	total: 1.24s	remaining: 50.5s
24:	learn: 1.4136200	total: 1.29s	remaining: 50.4s
25:	

In [ ]:
y_pred = model.predict(test_x)

In [ ]:
submission = pd.read_csv(ans)
submission['label'] = y_pred
submission.to_csv('./baseline_submission.csv', index=False)

In [ ]:
y_pred

array([[5],
       [0],
       [2],
       ...,
       [0],
       [2],
       [2]])

In [ ]:
#5 
#Cross Validation Scores: [0.5024975024975025, 0.48, 0.471, 0.464, 0.453]
#Average Accuracy: 0.47409950049950045